In [1]:
import os
# os.environ['HF_HOME'] = '/media/dong/storrage-01/huggingface'
print(os.getenv('HF_HOME'))

In [2]:
import argparse
import torch

from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
from llava.conversation import conv_templates, SeparatorStyle
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import tokenizer_image_token, get_model_name_from_path, KeywordsStoppingCriteria

from PIL import Image

import requests
from PIL import Image
from io import BytesIO
from transformers import TextStreamer, set_seed

In [3]:
# own
categories_own_reduction_of_smid = ", ".join([
"Drugs",
"Harm",
"Cruelty",
"Sexual"
"Exploitation",
"Inequality",
"Injustice",
"Degradation",
"Unfairness",
"Immoral actions",
#"None"
])

# chatgpt based on first run
categories_firstrun_chatgpt = ", ".join([ 
    "Violence and Harm",
    "Inappropriate or Offensive Behavior",
    "Sexual Content and Behavior",
    "Drug Use and Abuse",
    "Religious and Cultural Sensitivities",
    "Social and Political Issues",
    "Disasters and Emergencies",
    "Graphic and Disturbing Content" ,
    "Inappropriate Language and Gestures",
    "None"
])

# chatgpt based on first run
pos_and_neg_categories_firstrun_chatgpt = ", ".join([ 
    "Violence and Harm",
    "Inappropriate or Offensive Behavior",
    "Sexual Content and Behavior",
    "Drug Use and Abuse",
    "Religious and Cultural Sensitivities",
    "Social and Political Issues",
    "Disasters and Emergencies",
    "Graphic and Disturbing Content" ,
    "Inappropriate Language and Gestures",
    "Peace and Care",
    "Respectful and Appropriate Conduct",
    "Platonic Relationships and Respect for Boundaries",
    "Healthy Living and Sobriety",
    "Religious Tolerance and Cultural Respect",
    "Social Harmony and Political Cooperation",
    "Wellness and Safety Assurance",
    "Stability and Preparedness",
    "Wholesome and Comforting Content",
    "Polite Communication and Gestures",
])

pos_and_neg_categories_firstrun_chatgpt_wNA = ", ".join([ 
    "Violence and Harm",
    "Inappropriate or Offensive Behavior",
    "Sexual Content and Behavior",
    "Drug Use and Abuse",
    "Religious and Cultural Sensitivities",
    "Social and Political Issues",
    "Disasters and Emergencies",
    "Graphic and Disturbing Content" ,
    "Inappropriate Language and Gestures",
    "Peace and Care",
    "Respectful and Appropriate Conduct",
    "Platonic Relationships and Respect for Boundaries",
    "Healthy Living and Sobriety",
    "Religious Tolerance and Cultural Respect",
    "Social Harmony and Political Cooperation",
    "Wellness and Safety Assurance",
    "Stability and Preparedness",
    "Wholesome and Comforting Content",
    "Polite Communication and Gestures",
    "N/A"
])

current_prototype_test = ", ".join([ 
    '"None"',
    '"Peace or Care"',
    '"Respectful or Appropriate Content"',
    '"Polite Communication and Gestures"',
    '"Violence and Harm"',
    '"Explicit Sexual Content"',
    '"Nudity"',
    '"Suicide or Self Harm"',
    '"Animal Cruelty"',
    '"Guns or Illegal Weapons"',
    '"Drug Use or Abuse"',
    '"Illegal Activities"',
    '"Obscene gestures"', 
    '"Disasters"',
    '"Harassment"',
    '"Humiliation"',
    '"Discrimination"',
    #'"Misinformation"',
    '"Violation of Privacy"',
    '"Terrorism or Extremist content"',
])


In [4]:
def load_image(image_file):
    if image_file.startswith('http') or image_file.startswith('https'):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(image_file).convert('RGB')
    image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'].half().cuda()
    return image, image_tensor

In [5]:
def clear_conv(conv):
    conv.messages = []
    return conv

In [8]:
# model_path= "liuhaotian/llava-v1.5-7b"
model_path= "liuhaotian/llava-v1.5-13b"
# model_path= "liuhaotian/llava-v1.6-34b"
# model_path= "liuhaotian/llava-v1.6-mistral-7b"
model_base=None
num_gpus=1
# conv_mode=None
conv_mode=None
temperature=0.2
max_new_tokens=512
load_8bit=False
load_4bit=True
debug=False

In [9]:
disable_torch_init()

model_name = get_model_name_from_path(model_path)
# tokenizer, model, image_processor, context_len = load_pretrained_model(model_path, model_base, model_name, load_8bit, load_4bit)
tokenizer, model, image_processor, context_len = load_pretrained_model(model_path, model_base, model_name)

In [21]:
print('aa')

In [22]:
if 'llama-2' in model_name.lower():
    conv_mode = "llava_llama_2"
elif "llava-v1.6-34b" in model_name.lower():
    conv_mode = "chatml_direct"
# elif "llava-v1.6-mistral-7b" in model_name.lower():
#     conv_mode = "llava_v1"
elif "v1" in model_name.lower():
    conv_mode = "llava_v1"
elif "mpt" in model_name.lower():
    conv_mode = "mpt"
else:
    conv_mode = "llava_v0"


conv_ = conv_templates[conv_mode].copy()
if "mpt" in model_name.lower():
    roles = ('user', 'assistant')
else:
    roles = conv_.roles
print(roles)
print(conv_mode)


In [23]:
def run(image, conv, image_tensor, text=None, verbose=True):
    if text is not None:
        inp = text
    else:    
        try:
            inp = input(f"{roles[0]}: ")
        except EOFError:
            inp = ""
        if not inp:
            print("exit...")
    
    if image is not None:
        conv = clear_conv(conv)
        # first message
        if model.config.mm_use_im_start_end:
            inp = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN + '\n' + inp
        else:
            inp = DEFAULT_IMAGE_TOKEN + '\n' + inp
        conv.append_message(conv.roles[0], inp)
        image = None
    else:
        # later messages
        conv.append_message(conv.roles[0], inp)
    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt()

    if verbose:
        print(f"{roles[1]}: ", end="")
    input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()
    stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
    keywords = [stop_str]
    stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    attention_mask = input_ids.ne(0)
    
    with torch.inference_mode():
        output_ids = model.generate(
            input_ids,
            attention_mask=attention_mask,
            pad_token_id=tokenizer.eos_token_id,
            images=image_tensor,
            do_sample=True,
            temperature=0.2,
            max_new_tokens=1024,
            streamer=streamer if verbose else None,
            use_cache=True,
            stopping_criteria=[stopping_criteria])
    # outputs = tokenizer.decode(output_ids[0, input_ids.shape[1]:]).strip()
    outputs = tokenizer.decode(output_ids[0,:]).strip()
    conv.messages[-1][-1] = outputs
    return outputs

In [24]:
prompt_template = {
    'variant6_fixed_pos_and_neg_categories_chatgpt_1category_2score_LASTclassification_wBins': f"Assess the inappropriateness of the content and classify if it is compliant to policies." + ' Use the following json template to answer: {"content-category": '+f'str<{pos_and_neg_categories_firstrun_chatgpt_wNA}>'+', "score": str<Explicitly Inappropriate,Marginally Inappropriate,Neutral,Generally Acceptable,Fully Appropriate>}, "short-caption": str, "explanation": str, "final-assessment": str<Compliant/Review Needed>',
    'v7': f"Assess the inappropriateness of the content and classify if it is compliant to policies." +
' Use the following json template to answer: {"content-category": '+f'str<{current_prototype_test}>'+', "short-caption": str, "explanation": str, "score": str<"Explicitly Inappropriate", "Highly Inappropriate", "Moderately Inappropriate", "Barely Acceptable", "Generally Acceptable", "Suitable & Respectful", "Positive & Engaging", "Exemplary & Inspiring">}, "final-assessment": str<"Review Needed"/"Compliant">. A review is needed as soon as content is at least Marginally Inappropriate.',
}
print(prompt_template['v7'])

json_keys = {
    'variant6_fixed_pos_and_neg_categories_chatgpt_1category_2score_LASTclassification_wBins':
    ["content-category", "score", "short-caption", "explanation", "final-assessment"],
    'v7':
    ["content-category", "score", "short-caption", "explanation", "final-assessment"],
    'unlimited_categories_1category_2caption_3classification_wBins': 
    ["inappropriateness-category", "score", "short-caption", "explanation", "final-assessment"],
    
}

def parse_json(j, prompt_key):
    j = j.replace('</s>', '')
    j = j.replace('<s>', '')
    j = j.split('{')[1].split("}")[0]
    j = "{" + j + "}"
    # remove all '' and "" from json
    for e in json_keys[prompt_key]:
        if e not in j:
            raise ValueError(f'{e} missing in json. {j}')
        if e+'"' not in j:
            j = j.replace(e+':', f'"{e}":')
    return j
    
import warnings
def print_json(out, selected_prompt_template):
    try:
        out = parse_json(out, selected_prompt_template)
        json_object = json.loads(out)
        for k in json_object.keys():
            r = json_object[k]
            if k == 'final-assessment':
                r = r.replace('Review Needed', "\x1b[31mReview Needed\x1b[0m")
                r = r.replace('Compliant', "\x1b[32mCompliant \x1b[0m")
            elif k == 'score':
                if 'Inappropriate' in r:
                    r = f"\x1b[31m{r}\x1b[0m"
                else:
                    r = f"\x1b[32m{r}\x1b[0m"
            print(k+':', r)
    except Exception as e:
        raise e

In [25]:
def generate_single_response(image, image_tensor, text=None, verbose=True):
    if text is not None:
        inp = text
    else:
        inp = ""

    if image is not None:
        # first message
        if model.config.mm_use_im_start_end:
            inp = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN + '\n' + inp
        else:
            inp = DEFAULT_IMAGE_TOKEN + '\n' + inp

    input_ids = tokenizer_image_token(inp, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()
    attention_mask = input_ids.ne(0)
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    with torch.inference_mode():
        output_ids = model.generate(
            input_ids,
            attention_mask=attention_mask,
            pad_token_id=tokenizer.eos_token_id,
            images=image_tensor,
            do_sample=True,
            temperature=0.2,
            max_new_tokens=max_new_tokens,
            streamer=streamer if verbose else None,
            use_cache=True)
    outputs = tokenizer.decode(output_ids[0,:]).strip()
    return outputs

In [26]:
print(prompt_template['v7'])

In [27]:
import glob
import json
set_seed(1)
selected_prompt_template = "v7"
image_paths = glob.glob('demo/*.png')
print(f'Inference on {len(image_paths)} images')

In [28]:
image_, image_tensor_ = load_image('demo/warning.png')
image_.resize((512,512)).show()
for i in range(1):
    out = run(image=image_, conv=conv_, image_tensor=image_tensor_, text=prompt_template[selected_prompt_template], verbose=True)
    print_json(out, selected_prompt_template)

In [29]:
image_, image_tensor_ = load_image('demo/bomb.png')
image_.resize((512,512)).show()
for i in range(1):
    out = run(image=image_, conv=conv_, image_tensor=image_tensor_, text=prompt_template[selected_prompt_template], verbose=True)
    print_json(out, selected_prompt_template)

In [306]:
image_, image_tensor_ = load_image('demo/asperin.png')
image_.resize((512,512)).show()
for i in range(1):
    out = run(image=image_, conv=conv_, image_tensor=image_tensor_, text=prompt_template[selected_prompt_template], verbose=True)
    print_json(out, selected_prompt_template)

In [309]:
image_, image_tensor_ = load_image('demo/smoking.png')
image_.resize((512,512)).show()
for i in range(1):
    out = run(image=image_, conv=conv_, image_tensor=image_tensor_, text=prompt_template[selected_prompt_template], verbose=True)
    print_json(out, selected_prompt_template)

In [ ]:
image_, image_tensor_ = load_image('demo/kid_weed.png')
image_.resize((512,512)).show()
for i in range(1):
    out = run(image=image_, conv=conv_, image_tensor=image_tensor_, text=prompt_template[selected_prompt_template], verbose=True)
    print_json(out, selected_prompt_template)

In [ ]:
image_, image_tensor_ = load_image('demo/happy.png')
image_.resize((512,512)).show()
for i in range(1):
    out = run(image=image_, conv=conv_, image_tensor=image_tensor_, text=prompt_template[selected_prompt_template], verbose=True)
    print_json(out, selected_prompt_template)